# **Clase ternaria**

# *Importación de librerías*

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

# *Lectura de datos*

In [22]:
# from google.colab import drive
# drive.mount("/content/drive")

# DIR = "/content/drive/MyDrive/Maestría en Explotación de datos/Data Mining/TP de clasificación"
DIR = 'D:\Robin\DMEyF\datasets'

In [23]:
# Carga de df_train y creación de columna id
# df_dataset = pd.read_csv(f'{DIR}/competencia_01_crudo.csv')
# df_dataset.set_index('id', inplace=True)

# df_dataset = pd.read_csv(f'{DIR}/competencia_02_crudo.csv')
df_dataset = pd.read_csv(f'{DIR}/competencia_02_crudo.csv.gz')

df_dataset.head()

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,0.0,1.0,0.0,-44919.57,3.23,7136.0,24336.99,13.0,0.0,1466.25
1,249221468,201901,1,0,1,51,13,738.16,-325.09,491.31,...,0.0,1.0,0.0,-30184.75,15.24,393.0,16345.79,8.0,0.0,1548.36
2,249223005,201901,1,0,1,46,178,1014.31,9434.15,417.36,...,0.0,1.0,0.0,-3730.14,0.00,2755.0,1181.65,3.0,0.0,3917.82
3,249228180,201901,1,0,1,64,296,1028.33,6873.80,1129.43,...,0.0,1.0,0.0,-13196.25,0.00,8194.0,1892.04,1.0,0.0,2017.56
4,249232117,201901,1,0,1,77,349,11617.61,43291.55,10726.08,...,0.0,1.0,0.0,-22073.65,0.00,7876.0,21902.85,6.0,0.0,891.48


In [24]:
def asignar_clase_ternaria(df_dataset, anio_mes_cero):
    
    anio_mes_cero_date = pd.to_datetime(int(anio_mes_cero), format='%Y%m')
    anio_mes_uno_date = anio_mes_cero_date + pd.DateOffset(months=1)
    anio_mes_dos_date = anio_mes_cero_date + pd.DateOffset(months=2)
    anio_mes_uno = int(anio_mes_uno_date.strftime('%Y%m'))
    anio_mes_dos = int(anio_mes_dos_date.strftime('%Y%m'))

    df_mes0 = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_cero, ['numero_de_cliente', 'foto_mes']]
    df_mes0.set_index('numero_de_cliente', inplace=True)
    df_mes1 = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_uno, ['numero_de_cliente', 'foto_mes']]
    df_mes1.set_index('numero_de_cliente', inplace=True)
    df_mes2 = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_dos, ['numero_de_cliente', 'foto_mes']]
    df_mes2.set_index('numero_de_cliente', inplace=True)
        
    # Crear un DataFrame combinado con todos los índices
    all_clients = df_mes0.index.union(df_mes1.index).union(df_mes2.index)

    # Reindexar cada DataFrame para que tengan el mismo índice
    df_mes0_reindexed = df_mes0.reindex(all_clients)
    df_mes1_reindexed = df_mes1.reindex(all_clients)
    df_mes2_reindexed = df_mes2.reindex(all_clients)

    # Crear nuevas columnas para indicar la presencia en cada mes
    df_combined = pd.DataFrame(index=all_clients)
    df_combined['mes0'] = df_mes0_reindexed['foto_mes'].notna().astype(int)
    df_combined['mes1'] = df_mes1_reindexed['foto_mes'].notna().astype(int)
    df_combined['mes2'] = df_mes2_reindexed['foto_mes'].notna().astype(int)

    # Opcionalmente, si deseas mantener el formato de los DataFrames originales
    df_combined = df_combined.reset_index().rename(columns={'index': 'numero_de_cliente'})


    sum_mes1 = df_combined['mes1'].sum()
    sum_mes2 = df_combined['mes2'].sum()
    if sum_mes1 == 0 or sum_mes2 == 0:
        df_combined['clase_ternaria'] = np.nan
    
    else:
        # Definir las condiciones
        conditions = [
            (df_combined['mes0'] == 1) & (df_combined['mes1'] == 1) & (df_combined['mes2'] == 1),
            (df_combined['mes0'] == 1) & (df_combined['mes1'] == 1) & (df_combined['mes2'] == 0),
            (df_combined['mes0'] == 1) & (df_combined['mes1'] == 0) & (df_combined['mes2'] == 0)
        ]
        # Definir los valores que corresponden a cada condición
        values = ['CONTINUA', 'BAJA+2', 'BAJA+1']
        # Aplicar np.select para crear la nueva columna
        df_combined['clase_ternaria'] = np.select(conditions, values, default='OTRO CASO')

    # Crear un mapeo de numero_de_cliente a clase_ternaria
    cliente_a_clase = df_combined.set_index('numero_de_cliente')['clase_ternaria']

    # Asignar la clase_ternaria basada en el mapeo
    df_dataset.loc[df_dataset['foto_mes'] == anio_mes_cero, 'clase_ternaria'] = df_dataset.loc[df_dataset['foto_mes'] == anio_mes_cero, 'numero_de_cliente'].map(cliente_a_clase)

    return df_dataset

In [25]:
# df_dataset = asignar_clase_ternaria(df_dataset, 202101)
# df_dataset = asignar_clase_ternaria(df_dataset, 202102)
# df_dataset = asignar_clase_ternaria(df_dataset, 202103)
# df_dataset = asignar_clase_ternaria(df_dataset, 202104)
# df_dataset = asignar_clase_ternaria(df_dataset, 202105)
# df_dataset = asignar_clase_ternaria(df_dataset, 202106)

# Valores únicos de 'foto_mes' en df_dataset
valores_foto_mes = df_dataset['foto_mes'].unique()

# Asignar clase ternaria para cada mes
for foto_mes in valores_foto_mes:
    df_dataset = asignar_clase_ternaria(df_dataset, foto_mes)


In [26]:
df_dataset.groupby('foto_mes')['clase_ternaria'].value_counts().reset_index(name='count')

,foto_mes,clase_ternaria,count
0,201901,CONTINUA,123346
1,201901,BAJA+2,718
2,201901,BAJA+1,688
3,201902,CONTINUA,124389
4,201902,BAJA+1,720
...,...,...,...
112,202105,OTRO CASO,3
113,202106,CONTINUA,162889
114,202106,BAJA+2,1074
115,202106,BAJA+1,908


In [27]:
# df_dataset.loc[df_dataset['foto_mes'] == 202106]

In [28]:
# Exportar el DataFrame a un archivo CSV
# df_dataset.to_csv('dataset/competencia_01.csv', index=False)

### *Limpieza de datos*

In [9]:
# ##Eliminar las columnas cprestamos_personales y mprestamos_personales de df_dataset

# df_dataset.drop(columns=['cprestamos_personales', 'mprestamos_personales'], inplace=True)
# df_dataset

In [31]:
# ## Eliminar los registros que sean de la clase_ternaria 'BAJA+1', pero solo para aquellos meses diferentes a 202106
# print('Antes de eliminar BAJA+1')
# print(df_dataset['clase_ternaria'].value_counts())
# df_dataset = df_dataset.loc[~((df_dataset['clase_ternaria'] == 'BAJA+1') & (df_dataset['foto_mes'] != 202106))]
# print('\nDespués de eliminar BAJA+1')
# print(df_dataset['clase_ternaria'].value_counts())

df_dataset.to_csv(f'{DIR}/competencia_02_Sin_Baja1.csv.gz', compression='gzip')

In [11]:
# Exportar el DataFrame a un archivo CSV
# df_dataset.to_csv(f'{DIR}/competencia_01.csv')

# df_dataset.to_csv(f'{DIR}/competencia_02.csv')
# df_dataset.to_csv(f'{DIR}/competencia_02.csv.gz', compression='gzip')